In [13]:
using JuMP, GLPK
const MOI = JuMP.MathOptInterface

m = 3;
c = zeros(m,m);
#Make diag cost matrix c
for i in 1:m
    for j in 1:m
        if i == j
            c[i,i] = 0;
        end
        if j > i
            c[i,j] = c[j,i] = rand();
        end
        #print(c[i,j], " ")
    end
        #print("\n")
end
#t = 0:m-1;
model = JuMP.Model(JuMP.with_optimizer(GLPK.Optimizer))
@variable(model, x[1:m, 1:m], Bin)
@variable(model, t[1:m], Int)
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:m, j in 1:m))
#for i = 1:m
#        @constraint(model, t[i] >= 0)
#end
for i = 1:m, j = 2:m
    if i != j
        @constraint(model,t[j] >= t[i] + 1 - m*(1-x[i,j]))
    end
    @constraint(model, x[i,i] = 0)  
end
#@constraint(model, t[1]==0)
#@constraints(model, begin
        # Constraint 1 - Sum over j == 1
#        [i in 1:m, j in 2:m], sum(x[i,j])==1
        # Constraint 1 - Sum over i == 1
#        [i in 1:m, j in 2:m], sum(x[i,j])==1
#        end)
@constraint(model, const_a[i=1:m], sum(x[i,1:m])==1)
@constraint(model, const_b[j=1:m], sum(x[1:m,j])==1)

LoadError: In `@constraint(model)`: Not enough positional arguments

In [29]:
using JuMP, GLPK
const MOI = JuMP.MathOptInterface

m = 4;
c = zeros(m,m);
#Make diag cost matrix c
for i in 1:m
    for j in 1:m
        if i == j
            c[i,i] = 0;
        end
        if j > i
            c[i,j] = c[j,i] = rand();
        end
        #print(c[i,j], " ")
    end
        #print("\n")
end
#t = 0:m-1;
model = JuMP.Model(JuMP.with_optimizer(GLPK.Optimizer))
@variable(model, x[1:m, 1:m], Bin)
@variable(model, t[1:m], Int)
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:m, j in 1:m))
#for i = 1:m
#        @constraint(model, t[i] >= 0)
#end
for i = 1:m, j = 2:m
    if i != j
        @constraint(model,t[j] >= t[i] + 1 - m*(1-x[i,j]))
    end 
end
@constraint(model, const_a[i=1:m], x[i,i] == 0)
@constraint(model, const_b[i=1:m], sum(x[i,1:m])==1)
@constraint(model, const_c[j=1:m], sum(x[1:m,j])==1)
model
#Solve and test
JuMP.optimize!(model)
    term_status = JuMP.termination_status(model)
    primal_status = JuMP.primal_status(model)
    is_optimal = term_status == MOI.OPTIMAL

    # Check solution
    if is_optimal
        println("Objective is: ", JuMP.objective_value(model))
        println("Optimal solution is:")
        for i in 1:m
            for j in 1:m    
            print("x[$i,$j]=", JuMP.value(x[i,j]), " ")
            end
            print("\n")
        end
    else
        error("No se encontró una solución")
    end

Objective is: 2.1726852196779527
Optimal solution is:
x[1,1]=0.0 x[1,2]=1.0 x[1,3]=0.0 x[1,4]=0.0 
x[2,1]=0.0 x[2,2]=0.0 x[2,3]=0.0 x[2,4]=1.0 
x[3,1]=1.0 x[3,2]=0.0 x[3,3]=0.0 x[3,4]=0.0 
x[4,1]=0.0 x[4,2]=0.0 x[4,3]=1.0 x[4,4]=0.0 
